In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import os

In [3]:
!pip install keras-unet

In [4]:
import keras
from keras_unet.models import custom_unet

2023-03-06 12:16:15.943159: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
/home/kunal.jain001/miniconda3/envs/O/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [5]:
DATAPATH = '/raid/mpsych/RISTERLAB/fly_segmentation_experiments/classified_images/npy_files'
images_file = os.path.join(DATAPATH, 'fly-mar-06-original-images.npy')
labels_file = os.path.join(DATAPATH, 'fly-mar-06-segmented.npy')

In [6]:
images = np.load(images_file)
labels = np.load(labels_file)

In [7]:
images.shape

(56, 512, 512)

In [8]:
# full dataset does not have the last channel
images = images.reshape(images.shape[0],images.shape[1],images.shape[2],1)
labels = labels.reshape(labels.shape[0],labels.shape[1],labels.shape[2],1)

In [9]:
images.shape

(56, 512, 512, 1)

In [10]:
p = np.random.permutation(len(images))

In [11]:
images = images[p]
labels = labels[p]

In [12]:
labels = labels.astype(np.float64)

In [13]:
images = images.astype(np.float64)

for i in range(images.shape[0]):
    images[i] = (images[i] - images[i].min()) / (images[i].max() - images[i].min()) # normalize individually

In [14]:
images.shape

(56, 512, 512, 1)

In [21]:
X_train = images[0:40]
y_train = labels[0:40]
X_val = images[40:50]
y_val = labels[40:50]
X_test = images[50:]
y_test = labels[50:]

In [22]:
model = custom_unet(
    input_shape=(512, 512, 1),
    use_batch_norm=False,
    num_classes=1,
    filters=32,
    dropout=0.5,
    output_activation='sigmoid')

In [23]:
import keras.optimizers
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance

In [24]:
opt = keras.optimizer_v1.Adam(lr=0.01)

In [25]:
model.compile(optimizer = 'Adam',    
              loss='binary_crossentropy', 
              metrics=[iou, iou_thresholded])

In [ ]:
history = model.fit(X_train, 
                    y_train, 
                    32, 
                    epochs=100,
                    validation_data=(X_val, y_val), 
                    verbose=1)

Epoch 1/100
2/2 [==============================] - 231s 57s/step - loss: 0.6990 - iou: 0.0700 - iou_thresholded: 0.0456 - val_loss: 0.5409 - val_iou: 0.0931 - val_iou_thresholded: 6.9308e-04
Epoch 2/100
2/2 [==============================] - 229s 57s/step - loss: 0.4549 - iou: 0.0599 - iou_thresholded: 9.8619e-04 - val_loss: 0.4588 - val_iou: 0.0705 - val_iou_thresholded: 3.0662e-06
Epoch 3/100
2/2 [==============================] - 229s 57s/step - loss: 0.4177 - iou: 0.0361 - iou_thresholded: 7.4669e-06 - val_loss: 0.4405 - val_iou: 0.0620 - val_iou_thresholded: 3.0663e-06
Epoch 4/100
2/2 [==============================] - 229s 57s/step - loss: 0.4338 - iou: 0.0326 - iou_thresholded: 4.9775e-06 - val_loss: 0.4759 - val_iou: 0.0786 - val_iou_thresholded: 3.0663e-06
Epoch 5/100
2/2 [==============================] - 229s 57s/step - loss: 0.4142 - iou: 0.0506 - iou_thresholded: 3.8531e-06 - val_loss: 0.4885 - val_iou: 0.0836 - val_iou_thresholded: 3.0663e-06
Epoch 6/100
